# Chapter 2 -  Machine Learning Based Recommendation Systems
## Segment 2 - Model-based Collaborative Filtering Systems
## SVD Matrix Factorization

In [1]:
import numpy as np
import pandas as pd

import sklearn
from sklearn.decomposition import TruncatedSVD

The MovieLens dataset was collected by the GroupLens Research Project at the University of Minnesota. You can download the dataset for this demostration at the following URL: https://grouplens.org/datasets/movielens/100k/

### Preparing the data

In [2]:
#dataset is missing in original course data
columns = ['user_id', 'item_id', 'rating', 'timestamp']
#frame = pd.read_csv('ml-100k/u.data', sep='\t', names=columns)
frame = pd.read_csv('u.data', sep='\t', names=columns)
frame.head()

user_id  item_id  rating  timestamp
0      196      242       3  881250949
1      186      302       3  891717742
2       22      377       1  878887116
3      244       51       2  880606923
4      166      346       1  886397596

In [3]:
#dataset is missing in original course data
columns = ['item_id', 'movie title', 'release date', 'video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
          'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
          'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

# movies = pd.read_csv('ml-100k/u.item', sep='|', names=columns, encoding='latin-1')
movies = pd.read_csv('u.item', sep='|', names=columns, encoding='latin-1')
movie_names = movies[['item_id', 'movie title']]
movie_names.head()

item_id        movie title
0        1   Toy Story (1995)
1        2   GoldenEye (1995)
2        3  Four Rooms (1995)
3        4  Get Shorty (1995)
4        5     Copycat (1995)

In [4]:
combined_movies_data = pd.merge(frame, movie_names, on='item_id')
combined_movies_data.head()

user_id  item_id  rating  timestamp                 movie title
0      196      242       3  881250949                Kolya (1996)
1      186      302       3  891717742    L.A. Confidential (1997)
2       22      377       1  878887116         Heavyweights (1994)
3      244       51       2  880606923  Legends of the Fall (1994)
4      166      346       1  886397596         Jackie Brown (1997)

In [5]:
combined_movies_data.groupby('item_id')['rating'].count().sort_values(ascending=False).head()

item_id
50     583
258    509
100    508
181    507
294    485
Name: rating, dtype: int64

In [6]:
filter = combined_movies_data['item_id']==50
combined_movies_data[filter]['movie title'].unique()

array(['Star Wars (1977)'], dtype=object)

### Building a Utility Matrix

In [7]:
rating_crosstab = combined_movies_data.pivot_table(values='rating', index='user_id', columns='movie title', fill_value=0)
rating_crosstab.head()

movie title  'Til There Was You (1997)  1-900 (1994)  101 Dalmatians (1996)  \
user_id                                                                       
1                                  0.0           0.0                    2.0   
2                                  0.0           0.0                    0.0   
3                                  0.0           0.0                    0.0   
4                                  0.0           0.0                    0.0   
5                                  0.0           0.0                    2.0   

movie title  12 Angry Men (1957)  187 (1997)  2 Days in the Valley (1996)  \
user_id                                                                     
1                            5.0         0.0                          0.0   
2                            0.0         0.0                          0.0   
3                            0.0         2.0                          0.0   
4                            0.0         0.0                          0.0   
5                            0.0         0.0                          0.0   

movie title  20,000 Leagues Under the Sea (1954)  \
user_id                                            
1                                            3.0   
2                                            0.0   
3                                            0.0   
4                                            0.0   
5                                            0.0   

movie title  2001: A Space Odyssey (1968)  \
user_id                                     
1                                     4.0   
2                                     0.0   
3                                     0.0   
4                                     0.0   
5                                     4.0   

movie title  3 Ninjas: High Noon At Mega Mountain (1998)  \
user_id                                                    
1                                                    0.0   
2                                                    1.0   
3                                                    0.0   
4                                                    0.0   
5                                                    0.0   

movie title  39 Steps, The (1935)  ...  Yankee Zulu (1994)  \
user_id                            ...                       
1                             0.0  ...                 0.0   
2                             0.0  ...                 0.0   
3                             0.0  ...                 0.0   
4                             0.0  ...                 0.0   
5                             0.0  ...                 0.0   

movie title  Year of the Horse (1997)  You So Crazy (1994)  \
user_id                                                      
1                                 0.0                  0.0   
2                                 0.0                  0.0   
3                                 0.0                  0.0   
4                                 0.0                  0.0   
5                                 0.0                  0.0   

movie title  Young Frankenstein (1974)  Young Guns (1988)  \
user_id                                                     
1                                  5.0                3.0   
2                                  0.0                0.0   
3                                  0.0                0.0   
4                                  0.0                0.0   
5                                  4.0                0.0   

movie title  Young Guns II (1990)  Young Poisoner's Handbook, The (1995)  \
user_id                                                                    
1                             0.0                                    0.0   
2                             0.0                                    0.0   
3                             0.0                                    0.0   
4                             0.0                                    0.0   
5                             0.0                 

### Transposing the Matrix

In [8]:
rating_crosstab.shape

(943, 1664)

In [9]:
X = rating_crosstab.T
X.shape

(1664, 943)

### Decomposing the Matrix

In [10]:
SVD = TruncatedSVD(n_components=12, random_state=17)

resultant_matrix = SVD.fit_transform(X)

resultant_matrix.shape

(1664, 12)

### Generating a Correlation Matrix

In [11]:
corr_mat = np.corrcoef(resultant_matrix)
corr_mat.shape

(1664, 1664)

### Isolating Star Wars From the Correlation Matrix

In [12]:
movie_names = rating_crosstab.columns
movies_list = list(movie_names)

star_wars = movies_list.index('Star Wars (1977)')
star_wars

1398

In [13]:
corr_star_wars = corr_mat[1398]
corr_star_wars.shape

(1664,)

### Recommending a Highly Correlated Movie

In [14]:
list(movie_names[(corr_star_wars<1.0) & (corr_star_wars > 0.9)])


['Empire Strikes Back, The (1980)',
 'Men in Black (1997)',
 'Raiders of the Lost Ark (1981)',
 'Return of the Jedi (1983)',
 'Star Trek: First Contact (1996)',
 'Toy Story (1995)']

In [15]:
list(movie_names[(corr_star_wars<1.0) & (corr_star_wars > 0.95)])

['Return of the Jedi (1983)', 'Toy Story (1995)']